# Pixeltable / Label Studio Integration Demo

This tutorial demonstrates how to integrate Pixeltable with Label Studio, in order to provide seamless management of annotations data across the annotation workflow.

We'll assume that you're at least somewhat familiar with Pixeltable and have read the [Pixeltable Basics](https://pixeltable.github.io/pixeltable/tutorials/image-operations/) tutorial. You'll also need to have a Label Studio instance installed and running locally (or at some accessible URL).

This tutorial is more complex than the other Pixeltable tutorials, because it requires a running Label Studio instance to integrate with. It's best to run it in a local Pixeltable installation, rather than a Colab or Kaggle notebook.

## Configure Pixeltable

First we configure Pixeltable to communicate with Label Studio. You'll need to know the URL of your Label Studio installation (for a local install, it's usually http://localhost:8080/) and your API key (the "Access Token" under Account & Settings).

Then do one of two things. Either add the following lines to your `~/.pixeltable/config.yaml` file:
```
label_studio:
  url: my-label-studio-url
  api_key: my-label-studio-api-key
```
Or run the following command, substituting your URL and API key as appropriate.

In [ ]:
import os
os.environ['LABEL_STUDIO_URL'] = 'my-label-studio-url'
os.environ['LABEL_STUDIO_API_KEY'] = 'my-label-studio-api-key'

## Create the Base Table to Store Videos

Now we create the master table that will hold our videos to be annotated. This only needs to be done once, when we initially set up the workflow.

In [ ]:
import pixeltable as pxt

cl = pxt.Client()
cl.create_dir('ls_demo', ignore_errors=True)
schema = {
    'video': pxt.VideoType(),
    'date': pxt.TimestampType()
}
videos_table = cl.create_table('ls_demo.videos', schema)

## Populate It with Data

Now let's add some videos to the table to populate it. We'll add some local videos as well as a few that are scraped from the Internet. Notice that you can refer to input videos either by a local filename or a URL.

In [ ]:
from datetime import date
import glob

files = glob.glob('/Users/asiegel/Dropbox/workspace/pixeltable/local/sample-data/*.mp4')
files.append('s3://multimedia-commons/data/videos/mp4/ffe/ff3/ffeff3c6bf57504e7a6cecaff6aefbc9.mp4')
today = date(2022, 4, 22)
videos_table.insert({'video': file, 'date': today} for file in files)

Let's have a look at the table now.

In [ ]:
videos_table.show()

## Create a Label Studio project

Next we'll create a new Label Studio project and link it to a new view on the Pixeltable table. You can link a Label Studio project to either a base table or a view. For tables that are expecting a lot of input data, it's often easier to link to views.

In [ ]:
from pixeltable.datatransfer.label_studio import LabelStudioProject

# Create a view to filter on the specified date

videos_2024_04_22 = cl.create_view(
    'ls_demo.videos_2024_04_22',
    videos_table #,
    # filter=(videos_table.date == today)
)

# Add a column to the view to receive the annotations. The column
# that receives the annotations must always have the type
# `pxt.JsonType(nullable=True)`.

videos_2024_04_22.add_column(annotations=pxt.JsonType(nullable=True))

# Create a new Label Studio project that we will link to the view.
# We use the pixeltable `LabelStudioProject.create` method for this,
# which both creates the project in Label Studio and initializes a
# Pixeltable reference to it.

ls_project = LabelStudioProject.create(
    title='ls_demo.videos_2024_04_22',
    label_config='''
    <View> 
      <Video name="video" value="$video"/>
      <Text name="" value="License plate number" />
      <TextArea name="lpn" toName="video" placeholder=""/>
      <Text name="turn-label" value="Turn Type" />
      <Choices name="turn-type" toName="video" showInLine="true"> 
        <Choice value="left_turn"/> 
        <Choice value="right_turn"/>
        <Choice value="straight"/>
        <Choice value="unsure"/> 
      </Choices>
    </View>
    '''
)

# Now link the view and the project. This only needs to be done once:
# after the view and project are linked, the relationship is stored
# indefinitely in Pixeltable's metadata.

videos_2024_04_22.link_remote(ls_project)

If you look in the Label Studio UI now, you'll see that there's a new project with the name `ls_demo.videos_2022_04_22`. There are no tasks in it: all we've done is to specify a link between the Pixeltable view and Label Studio project; we haven't actually synchronized the data between them yet.

## Push Videos from the View to the Label Studio Project

Now let's actually push our videos from the view to the Label Studio project. Once the link has been established, all we need to do is call the handy `sync_remotes()` command on the view.

In [ ]:
videos_2024_04_22.sync_remotes()

Pixeltable remembers which videos have been synced, so if we call `sync_remotes()` again, it will only create new tasks for videos that have been added to the table since the last sync.

In [ ]:
videos_2024_04_22.sync_remotes()

## Some Time Later, Pull the Annotations

Now let's try pulling some annotations from Label Studio back to our view. As always, Pixeltable retains all its data and state in persistent storage, so we can safely clear the variables in our Jupyter notebook or restart the kernel, then pick up seamlessly where we left off.

At this point, try creating annotations for one of the videos using the Label Studio UI. Then come back to the notebook, restart your Jupyter kernel, and enter the following command.

In [ ]:
%reset -f
import pixeltable as pxt

cl = pxt.Client()
videos_2024_04_22 = cl.get_table('ls_demo.videos_2024_04_22')
videos_2024_04_22.sync_remotes()

Let's see what effect that had.

In [ ]:
videos_2024_04_22.select(videos_2024_04_22.video, videos_2024_04_22.annotations).show()

## Add a Computed Column

In [ ]:
videos_2024_04_22.add_column(turn_type=videos_2024_04_22.annotations[0].result[0].value.choices[0])

In [ ]:
videos_2024_04_22.select(videos_2024_04_22.video, videos_2024_04_22.annotations, videos_2024_04_22.turn_type).show()

## Frame Annotations

In [ ]:
# from pixeltable.iterators import FrameIterator

# frames = cl.create_view(
#     'ls_demo.frames_2024_04_22',
#     t,
#     iterator_class=FrameIterator,
#     iterator_args={'video': t.video, 'fps': 0.25}
# )
# Create a new Label Studio project and link it to the Table.



In [ ]:
frames.select(frames.frame).show()

In [ ]:
from pixeltable.datatransfer.label_studio import LabelStudioProject

remote = LabelStudioProject.create(
    'frames_2024_04_22',
    '''
    <View>
      <Image name="frame" value="$frame"/>
    </View>
    '''
)
